In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,time

In [2]:
# values must be sent in order: hour, minute?, second?
def int_to_hour(values,typeofvalues):
    if len(values) == len(typeofvalues):
        if 'hour' in typeofvalues:
            if 'minute'in typeofvalues:
                if 'second' in typeofvalues:
                    #pending
                    hour = pd.to_datetime(values[0],format="%H:%M:%S")
                else:
                    #pending
                    hour = pd.to_datetime(values[0],format="%H:%M")
            else:
                hour = pd.to_datetime(values[0],format="%H")
    return hour.strftime("%H:%M:%S")

In [37]:
dateCols = ['Start Timestamp','Complete Timestamp']
# Read CSV
df = pd.read_csv("CDM.csv", parse_dates=dateCols)
# Eliminate Assign Events
df = df[df['lifecycle:transition'] != 'assign']
# Add weekday to DataFrame
df['Day'] = df['Complete Timestamp'].dt.day_name()
df['Hour'] = df['Complete Timestamp'].dt.hour
df = df.drop(['Start Timestamp', 'Variant','Variant index','(case) creator','(case) variant','(case) variant-index','concept:name','org:resource','type_task'], axis=1)
df.rename(columns={'lifecycle:transition':'EventType','Complete Timestamp':'Timestamp'}, inplace=True)
resources = list(df['Resource'].unique())
backup_df = df
schedule_list = []
for resource in resources:
    day_list=[]
    starting_hours = []
    completing_hours = []
    hour_ranges = []
    df = backup_df[backup_df['Resource']==resource]
    min_list = np.array(df.groupby(['EventType','Resource','Day']).min()['Hour'].to_list())
    max_list = np.array(df.groupby(['EventType','Resource','Day']).max()['Hour'].to_list())
    grouped = df.groupby(['Resource','Day'])
    for name,group in grouped:
        day_list.append(name[1])
    for hour in min_list:
        r = int_to_hour([hour.item()],['hour'])
        starting_hours.append(r)
    for hour in max_list:
        r = int_to_hour([hour.item()],['hour'])
        completing_hours.append(r)
    ranges = list(zip(starting_hours, completing_hours))
    for tup in ranges:
        s,f = tup
        val = s + '-' + f
        hour_ranges.append(val)
    schedules = list(zip(day_list,hour_ranges)) + [resource]
    schedule_list.append(schedules)
schedule_list

[[('Friday', '00:00:00-23:00:00'),
  ('Monday', '00:00:00-23:00:00'),
  ('Saturday', '00:00:00-23:00:00'),
  ('Sunday', '00:00:00-23:00:00'),
  ('Thursday', '00:00:00-23:00:00'),
  ('Tuesday', '00:00:00-23:00:00'),
  ('Wednesday', '00:00:00-23:00:00'),
  'Start'],
 [('Monday', '13:00:00-13:00:00'), ('Wednesday', '16:00:00-16:00:00'), '8997'],
 [('Friday', '00:00:00-21:00:00'),
  ('Monday', '12:00:00-23:00:00'),
  ('Saturday', '00:00:00-17:00:00'),
  ('Thursday', '14:00:00-14:00:00'),
  ('Tuesday', '18:00:00-20:00:00'),
  ('Wednesday', '16:00:00-20:00:00'),
  '387'],
 [('Friday', '15:00:00-23:00:00'),
  ('Monday', '13:00:00-21:00:00'),
  ('Saturday', '00:00:00-07:00:00'),
  ('Thursday', '01:00:00-23:00:00'),
  ('Tuesday', '01:00:00-23:00:00'),
  ('Wednesday', '00:00:00-22:00:00'),
  '1007'],
 [('Friday', '00:00:00-23:00:00'),
  ('Monday', '01:00:00-23:00:00'),
  ('Saturday', '00:00:00-23:00:00'),
  ('Sunday', '05:00:00-21:00:00'),
  ('Thursday', '00:00:00-23:00:00'),
  ('Tuesday', '00:0